<a href="https://colab.research.google.com/github/sofiagpmg/Detecting-Cyberbully/blob/main/distilroberta_Simple_Transformer_20210415.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip3 install --upgrade transformers
!pip install simpletransformers

In [ ]:
#!pip3 install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm, notebook
from tqdm import tqdm


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/BERT_ICD10 classification/mtcompare_m3.csv',encoding = 'iso-8859-1',names=['id', 'text', 'ICD_Chapter', 'label'])
df.head()

,id,text,ICD_Chapter,label
0,id,text,ICD_Chapter,label
1,PAT001,subjective this 23-year-old white female prese...,Diseases of the respiratory system,1
2,PAT002,history a 34-year-old male presents today self...,Diseases of the respiratory system,1
3,PAT003,chief complaint this 5-year-old male presents ...,Diseases of the respiratory system,1
4,PAT004,history i had the pleasure of meeting and eval...,Diseases of the respiratory system,1


In [ ]:
df.columns

Index(['id', 'text', 'ICD_Chapter', 'label'], dtype='object')

In [ ]:
df = df.iloc[1:]

In [ ]:
df.head()

,id,text,ICD_Chapter,label
1,PAT001,subjective this 23-year-old white female prese...,Diseases of the respiratory system,1
2,PAT002,history a 34-year-old male presents today self...,Diseases of the respiratory system,1
3,PAT003,chief complaint this 5-year-old male presents ...,Diseases of the respiratory system,1
4,PAT004,history i had the pleasure of meeting and eval...,Diseases of the respiratory system,1
5,PAT005,history a 55-year-old female presents self-ref...,Diseases of the respiratory system,1


In [ ]:
df = df[['text', 'label']]

In [ ]:
df.label = pd.to_numeric(df.label, errors='coerce')
print(df.dtypes)

text     object
label     int64
dtype: object


In [ ]:
sorted(df.label.unique())
num_labels = df.label.nunique()
num_labels

7

In [ ]:
possible_labels = df.label.unique()
print(possible_labels)

[1 4 0 6 5 2 3]


In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
df['label'] = df.label.replace(label_dict)

In [ ]:
from sklearn.model_selection import train_test_split
#target = 'label'

train_df, eval_df = train_test_split(df, test_size=0.2,stratify=df.label.values)

In [ ]:

from simpletransformers.classification import ClassificationModel


# Create a ClassificationModel
model = ClassificationModel('roberta', 'distilroberta-base', num_labels=num_labels, use_cuda=False, args={'overwrite_output_dir':True})

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
%%time
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2992 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/374 [00:00<?, ?it/s]

In [ ]:
%%time
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)


In [ ]:
result

In [ ]:
model_outputs

In [ ]:
_, predictions, true_vals = model.eval_model(eval_df)

In [ ]:
from pandas import DataFrame
your_list = [predictions, true_vals]
df = DataFrame (your_list)


In [ ]:
df.head()

In [ ]:
df.to_csv('check.csv')

In [ ]:
predictions = []
for x in model_outputs:
    predictions.append(np.argmax(x))

#print('f1 score:', f1_score(eval_df['label'], predictions,average='micro'))

#target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(eval_df['label'], predictions,sorted(possible_labels)))#,target_names=sorted(df['label'].unique())))


In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(eval_df['label'], predictions,sorted(possible_labels))

In [ ]:
import seaborn as sns
sns.heatmap(cf, annot=True)

In [ ]:
sns.set(font_scale=1.4) # for label size
sns.heatmap(cf, annot=True, annot_kws={"size": 14}, cmap='Blues') # font size


In [ ]:
eval_df.to_csv('eval.csv')

In [ ]:
pred = pd.DataFrame(predictions, eval_df)
pred.to_csv('pred_chk.csv')

In [ ]:
#from sklearn import metrics
#print(metrics.classification_report(eval_df['label'], predictions, target_names=df['label'].unique()))

In [ ]:
#model.train_model(train_df)

#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

#lst = []
#for arr in model_outputs:
#    lst.append(np.argmax(arr))
#true = eval_df['label'].tolist()
#predicted = lst
#sklearn.metrics.accuracy_score(true,predicted)

#https://stackoverflow.com/questions/64595546/higher-accuracy-using-simpletransformers-vs-transformers-library-with-bert

In [ ]:
#print(metrics.classification_report(true_vals, predictions, target_names=df['label'].unique()))
#https://github.com/shudima/notebooks/blob/master/BERT_to_the_rescue.ipynb


#model.train_model(train_df)

#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

#lst = []
#for arr in model_outputs:
#    lst.append(np.argmax(arr))
#true = eval_df['label'].tolist()
#predicted = lst
#sklearn.metrics.accuracy_score(true,predicted)